In [1]:
import os

import numpy as np
import pandas as pd
import seaborn as sns

from ray import init, rllib, shutdown

In [2]:
from attack_simulator.agents import ATTACKERS
from attack_simulator.env import AttackSimulationEnv
from attack_simulator.graph import AttackGraph, SIZES

In [3]:
class NoAction(rllib.policy.Policy):
    def compute_actions(self, observations, *args, **kwargs):
        # FIXME: use a `numpy` array as a temporary workaround for
        #        https://github.com/ray-project/ray/issues/10100
        return np.zeros(len(observations)), [], {}

    def get_weights(self):
        return {}

    def set_weights(self, weights):
        pass


no_action = rllib.agents.trainer_template.build_trainer(name="NoAction", default_policy=NoAction)

In [4]:
# work around: https://github.com/ray-project/ray/issues/17618

from IPython.core.interactiveshell import InteractiveShell


class keep_ipython_sane:
    def __enter__(self):
        self.instance = InteractiveShell.instance()

    def __exit__(self, *args, **kwargs):
        # feel free to improve with error handling, etc.
        InteractiveShell._instance = self.instance

In [5]:
if os.path.isdir("/var/run/secrets/kubernetes.io"):  # inside k8s pod
    args = dict(address="auto")
else:
    # listen on all interfaces inside a container for port-forwarding to work
    dashboard_host = "0.0.0.0" if os.path.exists("/.dockerenv") else "127.0.0.1"
    args = dict(num_cpus=4, dashboard_host=dashboard_host)

# ALTERNATIVE: use the "Ray client" to connect to a remote cluster
# Unfortunately, JupyterNotebookReporter displays an object reference
# <IPython.core.display.HTML object> instead of content...
# --- --- ---
#
# from ray.util.client import worker
#
# worker.INITIAL_TIMEOUT_SEC = worker.MAX_TIMEOUT_SEC = 1
#
# ray_client_server = 'host.docker.internal' if os.path.exists("/.dockerenv") else '127.0.0.1'
# try:
#     init(address=f'ray://{ray_client_server}:10001')
# except Connection Error:
#     pass  # TODO: try something else...

In [6]:
from tqdm.auto import tqdm

rename = dict(
    attacker="Attacker",
    graph_size="Graph size",
    episode_reward="Returns",
    episode_lengths="Episode lengths",
)

num_episodes = 5  # run 5 episodes on the same environment
attackers = list(ATTACKERS)
graphs = [AttackGraph(dict(graph_size=size)) for size in SIZES]
seeds = [0, 1, 2, 3, 6, 7, 11, 28, 42, 1337]


def generate(savename):
    init(**args)

    frames = []
    for seed in tqdm(seeds, "seeds"):
        
        for attacker in tqdm(attackers, f"└── {seed}"):
            for graph in tqdm(graphs, f"\u00a0\u2001\u2001\u2001└── {attacker}@{seed}"):
                config = dict(
                    log_level="ERROR",
                    framework="torch",
                    env=AttackSimulationEnv,
                    env_config=dict(attack_graph=graph, attacker=attacker),
                    seed=seed,
                    rollout_fragment_length=1,
                    evaluation_interval=1,
                    evaluation_num_workers=1,
                    evaluation_config=dict(explore=False),
                    evaluation_num_episodes=num_episodes,
                )
                with keep_ipython_sane():
                    agent = no_action(config=config)
                try:
                    stats = agent.evaluate()["evaluation"]["hist_stats"]
                    frame = pd.DataFrame(dict(attacker=attacker, graph_size=graph.num_attacks, **stats))
                    frames.append(frame)
                except Exception as e:
                    print(seed, attacker, graph.graph_size, e)
    shutdown()
    results_df = pd.concat(frames, ignore_index=True).rename(columns=rename)
    results_df.to_csv(savename)
    return results_df

AttackGraph(en2720.yaml[tiny], 2 services, 7 attack steps)
AttackGraph(en2720.yaml[small], 5 services, 27 attack steps)
AttackGraph(en2720.yaml[medium-small], 9 services, 44 attack steps)
AttackGraph(en2720.yaml[medium], 13 services, 56 attack steps)
AttackGraph(en2720.yaml[large], 16 services, 62 attack steps)
AttackGraph(en2720.yaml[extra-large], 16 services, 65 attack steps)
AttackGraph(en2720.yaml[full], 18 services, 78 attack steps)


In [7]:
savename = "length-agent-eval.csv"

df = generate(savename) if not os.path.exists(savename) else pd.read_csv(savename, index_col=0)

2021-10-10 15:16:23,472	INFO worker.py:826 -- Connecting to existing Ray cluster at address: 10.28.0.20:6379


seeds:   0%|          | 0/10 [00:00<?, ?it/s]

└── 0:   0%|          | 0/6 [00:00<?, ?it/s]

    └── well-informed@0:   0%|          | 0/7 [00:00<?, ?it/s]

2021-10-10 15:16:23,764	INFO trainer.py:758 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


...

    └── random@0:   0%|          | 0/7 [00:00<?, ?it/s]

└── 1:   0%|          | 0/6 [00:00<?, ?it/s]

    └── well-informed@1:   0%|          | 0/7 [00:00<?, ?it/s]

2021-10-10 15:30:50,298	WARNING worker.py:1227 -- The node with node id: d98e297cc2983982fdafad5880a20935ab45840526706ae96c3d2997 and ip: 10.28.0.73 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a raylet crashes unexpectedly or has lagging heartbeats.
2021-10-10 15:33:14,477	WARNING worker.py:1227 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff86b46b897f8a9098b8b080b901000000 Worker ID: 8e52485a83783d4cde5789dbc4a8192a1f589c69ba1fbe886b8f4d2a Node ID: 1715b0f1fe8add476cafb7dac8d71f7fd523f6a7d2c9cb3d46b2264a Worker IP address: 10.28.0.20 Worker port: 10075 Worker PID: 19339


NameError: name 'RayActorError' is not defined

In [ ]:
df

In [ ]:
sns.set(style="darkgrid", rc={"figure.figsize": (12, 8)})

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Returns", hue="Attacker", ci="sd")
g.legend(title="Attacker", loc="upper right")
g.set_title("Defender: no-action")

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Episode lengths", hue="Attacker", ci="sd")
g.legend(title="Attacker", loc="upper left")
g.set_title("Defender: no-action")

In [ ]:
pd.set_option("display.max_columns", 32)
df.groupby("Attacker").describe()